In [1]:
# import packages 导入库，搭建环境
import numpy as np
import pandas as pd  
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# ignore warnings 可以删除
import warnings
warnings.filterwarnings('ignore')

# 为了让matplotlib里显示中文字符
import matplotlib.font_manager as fm 
matplotlib.rcParams['font.family'] = 'Microsoft Yahei' # 设置字体

In [36]:
df = pd.read_csv("fraud_oracle.csv")

In [37]:
df.head().T

,0,1,2,3,4
Month,Dec,Jan,Oct,Jun,Jan
WeekOfMonth,5,3,5,2,5
DayOfWeek,Wednesday,Wednesday,Friday,Saturday,Monday
Make,Honda,Honda,Honda,Toyota,Honda
AccidentArea,Urban,Urban,Urban,Rural,Urban
DayOfWeekClaimed,Tuesday,Monday,Thursday,Friday,Tuesday
MonthClaimed,Jan,Jan,Nov,Jul,Feb
WeekOfMonthClaimed,1,4,2,1,2
Sex,Female,Male,Male,Male,Female
MaritalStatus,Single,Single,Married,Married,Single


In [38]:
df.columns= ['事故月份',
            '事故月中周数',
            '事故周中天数',
            '品牌',
            '事故发生区域',
            '出险周中天数',
            '出险月份',
            '出险月中周数',
            '出险人性别',
            '出险人婚姻状况',
            '出险人年龄',
            '责任划分',
            '保单类别',
            '车辆类别',
            '车辆价格',
            '是否骗保',
            '保单编号',
            '客服编号',
            '起付额',
            '驾驶员评级',
            '事故距离保单生效天数',
            '出险距离保单生效天数',
            '历史出险次数',
            '车龄',
            '投保人年龄',
            '是否出警',
            '是否有目击者',
            '中介类型',
            '补充出险次数',
            '出险距离地址变动时间',
            '投保人车辆数',
            '事故年份',
            '车险类别']

In [39]:
df[['事故月份', '出险月份']] = df[['事故月份', '出险月份']].replace({'Dec': '十二月',
                                                                'Nov': '十一月',
                                                                'Oct': '十月',
                                                                'Sep': '九月',
                                                                'Aug': '八月',
                                                                'Jul': '七月',
                                                                'Jun': '六月',
                                                                'May': '五月',
                                                                'Apr': '四月',
                                                                'Mar': '三月',
                                                                'Feb': '二月',
                                                                'Jan': '一月'})

In [40]:
df[['事故周中天数', '出险周中天数']] = df[['事故周中天数', '出险周中天数']]\
    .replace({'Monday': '周一',
            'Tuesday': '周二',
            'Wednesday': '周三',
            'Thursday': '周四',
            'Friday': '周五',
            'Saturday': '周六',
            'Sunday': '周日',})

In [41]:
df['品牌'] = df['品牌'].replace({'Honda': '本田',
                                'Toyota': '丰田',
                                'Ford': '福特',
                                'Mazda': '马自达',
                                'Chevrolet': '雪佛兰',
                                'Pontiac': '庞蒂亚克',
                                'Accura': '讴歌',
                                'Dodge': '道奇',
                                'Mercury': '别克',
                                'Jaguar': '捷豹',
                                'Nisson': '日产',
                                'VW': '大众',
                                'Saab': '萨博',
                                'Saturn': '现代',
                                'Porche': '保时捷',
                                'BMW': '宝马',
                                'Mecedes': '奔驰',
                                'Ferrari': '法拉利',
                                'Lexus': '凌志'})

In [44]:
df['事故发生区域'] = df['事故发生区域'].replace({'Urban': '城区', 'Rural': '农村'})

In [46]:
df['出险人性别'] = df['出险人性别'].replace({'Male': '男', 'Female': '女'})

In [49]:
df['出险人婚姻状况'] = df['出险人婚姻状况'].replace({'Single': '单身',
                                                'Married': '有配偶',
                                                'Widow': '丧偶',
                                                'Divorced': '离异'})

In [51]:
df['责任划分'] = df['责任划分'].replace({'Policy Holder': '出险人', 'Third Party': '第三方'})

In [52]:
df.head().T

,0,1,2,3,4
事故月份,十二月,一月,十月,六月,一月
事故月中周数,5,3,5,2,5
事故周中天数,周三,周三,周五,周六,周一
品牌,本田,本田,本田,丰田,本田
事故发生区域,城区,城区,城区,农村,城区
出险周中天数,周二,周一,周四,周五,周二
出险月份,一月,一月,十一月,七月,二月
出险月中周数,1,4,2,1,2
出险人性别,女,男,男,男,女
出险人婚姻状况,单身,单身,有配偶,有配偶,单身


In [2]:
def missingRateIndex(data,thre=0.95):
    data = np.array(data)
    nrow = data.shape[0]
    if nrow == 0:
            nrow = 1
    #print nrow
    miss_index = []
    for i in range(np.shape(data)[1]):
        #con1 = np.any([data[:,i] == ' ', data[:,i] == 'NA'],axis=1)
        #missing_num = data[con1,0].shape[0]
        missing_num = data[np.any([data[:,i] == '', data[:,i] == 'NA', data[:,i] == 'NULL'],axis=0),i].shape[0]
        if round(missing_num/float(nrow),3) >= thre:
            miss_index.append(i)
    return miss_index
